In [1]:
from bitcoinutils.setup import setup
from bitcoinutils.keys import P2shAddress, PublicKey
from bitcoinutils.script import Script
from bitcoinutils.utils import to_satoshis
from bitcoinutils.transactions import Transaction, TxInput, TxOutput, TxWitnessInput
from bitcoinutils.keys import P2pkhAddress, P2shAddress, P2wshAddress, PrivateKey, PublicKey, Address
from bitcoinutils.proxy import NodeProxy
import hashlib
from enum import Enum
from bitcoinutils.setup import setup
from binascii import unhexlify, hexlify

In [2]:
setup('regtest')
private_key1 = PrivateKey("cTicJXw3ywZptXupB453nmPCznnueXygw8vw1ckyAkQ63RZc7d34")
private_key2 = PrivateKey("cRQrsYJo8MPPkevELzJ85b3ei3M7p1doFfWv6oMQeFGupxFrq1sv")

In [3]:
# Method - 1
p2wsh_witness_script = Script(['OP_2', private_key1.get_public_key().to_hex(), private_key2.get_public_key().to_hex(),'OP_2', 'OP_CHECKMULTISIG'])
script_bytes = p2wsh_witness_script.to_bytes()
hashsha256 = hashlib.sha256(script_bytes).digest()
hashsha256 = hexlify(hashsha256).decode('utf-8')
p2sh_redeem_script = Script(['OP_0', hashsha256])
p2sh_address = P2shAddress.from_script(p2sh_redeem_script)
print(p2sh_address.to_string())
print(p2sh_address.to_script_pub_key())

2N4vHiXTCCVMt4TYimTGXRCySzNLXkdiPyH
['OP_HASH160', '800cb7e6de5fbf5520d437be4d0cbf8af82054b3', 'OP_EQUAL']


In [4]:
# # Method - 2
# txin_redeemScript = Script(['OP_2', private_key1.get_public_key().to_hex(), private_key2.get_public_key().to_hex(),'OP_2', 'OP_CHECKMULTISIG'])
# print(txin_redeemScript.to_p2wsh_script_pub_key())
# p2wsh_address = P2wshAddress.from_script(txin_redeemScript)
# print(p2wsh_address.to_string())
# p2sh_address = P2shAddress.from_script(p2wsh_address.to_script_pub_key())
# print(p2sh_address.to_string())

In [5]:
sk1 = PrivateKey('cTicJXw3ywZptXupB453nmPCznnueXygw8vw1ckyAkQ63RZc7d34')
sk2 = PrivateKey('cRQrsYJo8MPPkevELzJ85b3ei3M7p1doFfWv6oMQeFGupxFrq1sv')

In [8]:
proxy = NodeProxy('deep', 'mdsSgY3ZUXe6x5JIQ81gpNwPPDu-arnaRGh-AGmi0IM').get_proxy()

# p2pkh_addr = P2pkhAddress('n4bkvTyU1dVdzsrhWBqBw8fEMbHjJvtmJR')
n_blocks = proxy.getblockcount()
n_mempool = proxy.getmempoolinfo()["size"]
total_txs = sum(
    [proxy.getblockstats(block, ["txs"])["txs"] for block in range(1, n_blocks)]
)
all_txs = proxy.listtransactions("*", total_txs + n_mempool, 0, True)
p2sh_txs = [*filter(lambda tx: tx["address"] == p2sh_address.to_string(), all_txs)]
print(p2sh_txs, end='\n\n')

p2sh_utxo = []
amount_to_send = 0.0
MIN_CONF = 0

for tx in p2sh_txs:
    if not proxy.gettxout(tx["txid"], tx["vout"]):
        print("tx [{}] already spent".format(tx["txid"]))
        continue
    if int(tx["confirmations"]) >= MIN_CONF and tx["category"] == "send":
        tx_in = TxInput(tx["txid"], tx["vout"])
        print(
            "*Found UTXO with {0:.8f} tBTC, TXID : [{1}]".format(
                abs(tx["amount"]), tx["txid"]
            )
        )
        p2sh_utxo += [tx_in]
        amount_to_send += abs(float(tx["amount"]))

if not p2sh_utxo:
    raise RuntimeError("No UTXO's currently associated with that P2SH address")

PRICE_PER_BYTE = 0.00078 / 1024
tx_fee = (len(p2sh_utxo) * 148 + 1 * 34 + 10) * PRICE_PER_BYTE

print("Total spendable amount aggregated from {0} UTXO(s) associated with P2SH address: {1:.8f} tBTC".format(len(p2sh_utxo), amount_to_send))
print("TX fee in satoshis: {0} (Fee Rate is {1:.8f} tBTC/KB)".format(to_satoshis(tx_fee), 0.00078))
amount_to_send -= tx_fee

print("Sending {0} satoshis ({1:.8f} BTC) to P2PKH address [{2}]".format(to_satoshis(amount_to_send), amount_to_send, p2pkh_addr.to_string()))

txout = TxOutput(to_satoshis(amount_to_send), p2pkh_addr.to_script_pub_key())
tx = Transaction(inputs=p2sh_utxo, outputs=[txout], has_segwit=False)
print("\nRaw transaction:\n" + tx.serialize())

[{'address': '2N4vHiXTCCVMt4TYimTGXRCySzNLXkdiPyH', 'category': 'send', 'amount': Decimal('-0.10000000'), 'vout': 1, 'fee': Decimal('-0.00001430'), 'confirmations': 0, 'trusted': True, 'txid': 'f5f9c98b0c7b0731cba11f5a1b0494a62d2a2de5c09aaf3ac37fa56d48c271bd', 'wtxid': 'bed63897ca9c23ee891a9752b143ee063bf9aa97c813c5a455210a5f8437be6a', 'walletconflicts': [], 'time': 1693271904, 'timereceived': 1693271904, 'bip125-replaceable': 'yes', 'abandoned': False}, {'address': '2N4vHiXTCCVMt4TYimTGXRCySzNLXkdiPyH', 'category': 'send', 'amount': Decimal('-0.10000000'), 'vout': 0, 'fee': Decimal('-0.00001430'), 'confirmations': 0, 'trusted': True, 'txid': '7d1ca2f8ec816e19716a9fd462fb4b124d510edf9dd5cf0d6d115aca64c5f74d', 'wtxid': '9c7d1033fcbd2a552e334081e353a50bc77389dedef9c0cc9f3509ed5d2a62a4', 'walletconflicts': [], 'time': 1693388894, 'timereceived': 1693388894, 'bip125-replaceable': 'yes', 'abandoned': False}]

*Found UTXO with 0.10000000 tBTC, TXID : [f5f9c98b0c7b0731cba11f5a1b0494a62d2a2de5

In [9]:
# txin_redeemScript = Script(['OP_2', private_key1.get_public_key().to_hex(), private_key2.get_public_key().to_hex(),'OP_2', 'OP_CHECKMULTISIG'])
# txin_scriptPubKey = Script(['OP_0', hashsha256])

for tx_idx, txin in enumerate(p2sh_utxo):
    sig1 = sk1.sign_input(tx, tx_idx, p2sh_redeem_script)
    sig2 = sk2.sign_input(tx, tx_idx, p2sh_redeem_script)
    
    # unlocking_script = Script(['OP_0', sig1, sig2, p2sh_redeem_script.to_hex()])
    witness_data = [b'', sig1, sig2, p2wsh_witness_script.to_hex()]
    tx.witnesses.append(TxWitnessInput(witness_data))
    unlocking_script = Script(['OP_0', p2sh_redeem_script.to_hex()])
    txin.script_sig = unlocking_script


In [10]:
print(tx, end='\n\n\n')
final_signed_tx = tx.serialize()

# display raw signed transaction ready to be broadcasted
print("\nRaw signed transaction: %s\n" % final_signed_tx)
print("=" * 71)

# display the transaction id
print("Tx ID: %s" % tx.get_txid())
print("=" * 71)

# confirm the validity of the transaction by performing a mempool acceptance test
response_status = proxy.testmempoolaccept([final_signed_tx])[0]

{'inputs': [{'txid': 'f5f9c98b0c7b0731cba11f5a1b0494a62d2a2de5c09aaf3ac37fa56d48c271bd', 'txout_index': 1, 'script_sig': ['OP_0', '0020823dfa1e4f88d20a6d3d35eed58ec16d87da30b3da1e87910803f617d76be223']}, {'txid': '7d1ca2f8ec816e19716a9fd462fb4b124d510edf9dd5cf0d6d115aca64c5f74d', 'txout_index': 0, 'script_sig': ['OP_0', '0020823dfa1e4f88d20a6d3d35eed58ec16d87da30b3da1e87910803f617d76be223']}], 'outputs': [{'amount': 19974102, 'script_pubkey': ['OP_DUP', 'OP_HASH160', 'fd337ad3bf81e086d96a68e1f8d6a0a510f8c24a', 'OP_EQUALVERIFY', 'OP_CHECKSIG']}], 'has_segwit': False, 'witnesses': [{'witness_items': [b'', '30440220383d3efc04fb5f6573ece8950872707fd1527e59a21f1c413cd050e692f6c3d40220527ec9e8374fe966da95269f18b2b9bac55915383a3b1b41963ee35225ad69b001', '30440220423354eda6291012ede20adb9649615b2204049fcf47c619f5bf45d6bba2a088022049d1f708d0109ffd1b895cbeb961df0c25f1962da74971869b5a55057ba237c501', '52210283aacf4507f578596fc8d65f393ecf8bc38a9740035c267bf745f89b4497626b210303db6694f92b732d39e7eb

In [11]:
response_status

{'txid': '54ea990d7c35dbe548b8bb50be8a7b74226d05fb010cef96c642680055cb876f',
 'wtxid': '54ea990d7c35dbe548b8bb50be8a7b74226d05fb010cef96c642680055cb876f',
 'allowed': False,
 'reject-reason': 'non-mandatory-script-verify-flag (Witness requires only-redeemscript scriptSig)'}